# Large Datasets

## Fine-Tuning

In [ ]:
import numpy as np
import tensorflow as tf

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, features_path, labels_path, batch_size):
        self.features = np.load(features_path)
        self.labels = np.load(labels_path)
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.features))

    def __len__(self):
        return int(np.ceil(len(self.features) / self.batch_size))

    def __getitem__(self, index):
        start = index * self.batch_size
        end = (index + 1) * self.batch_size
        batch_features = self.features[start:end]
        batch_labels = self.labels[start:end]
        return batch_features, batch_labels

# Example usage:
train_features_path = "train_features.npy"
train_labels_path = "train_labels.npy"
batch_size = 4096

train_generator = DataGenerator(train_features_path, train_labels_path, batch_size)

In [ ]:
CBLANE = load_model("CBLANEFT.keras")
CBLANE.compile(loss='binary_crossentropy',
                             optimizer=Adam(learning_rate=0.001),
                             metrics=[BinaryAccuracy(),
                                      AUC()]
                             )
history = CBLANE.fit(train_generator,
                             epochs=30,
                             verbose=1,
                             callbacks=([
                                         SaveSubModels()])
                             )


Epoch 1/30
   6/3519 [..............................] - ETA: 36:22 - loss: 0.4004 - binary_accuracy: 0.8142 - auc: 0.8986

3519/3519 [==============================] - 2301s 648ms/step - loss: 0.2629 - binary_accuracy: 0.8887 - auc: 0.9572
Epoch 2/30
3519/3519 [==============================] - 2275s 647ms/step - loss: 0.2282 - binary_accuracy: 0.9052 - auc: 0.9678
Epoch 3/30
3519/3519 [==============================] - 2275s 647ms/step - loss: 0.2174 - binary_accuracy: 0.9103 - auc: 0.9707
Epoch 4/30
2224/3519 [=================>............] - ETA: 13:57 - loss: 0.2113 - binary_accuracy: 0.9132 - auc: 0.9723

## Pre Fine-Tuning

In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for test_feature,test_label in zip(test_features,test_labels):
  i=i+1
  print("Dataset:",i)
  data = CBLANE.evaluate(test_feature,test_label,verbose=1,batch_size=4096)
  avg_acc.append(data[2])
print(np.mean(avg_acc))

Dataset: 1
3/3 [==============================] - 8s 348ms/step - loss: 0.6136 - binary_accuracy: 0.7261 - auc: 0.8609
Dataset: 2
5/5 [==============================] - 2s 325ms/step - loss: 0.1572 - binary_accuracy: 0.9421 - auc: 0.9869
Dataset: 3
7/7 [==============================] - 2s 355ms/step - loss: 0.3625 - binary_accuracy: 0.8385 - auc: 0.9365
Dataset: 4
2/2 [==============================] - 2s 2s/step - loss: 0.1841 - binary_accuracy: 0.9269 - auc: 0.9785
Dataset: 5
4/4 [==============================] - 1s 197ms/step - loss: 0.3530 - binary_accuracy: 0.8424 - auc: 0.9308
Dataset: 6
2/2 [==============================] - 2s 1s/step - loss: 0.3629 - binary_accuracy: 0.8458 - auc: 0.9294
Dataset: 7
2/2 [==============================] - 2s 2s/step - loss: 0.6040 - binary_accuracy: 0.7336 - auc: 0.8494
Dataset: 8
3/3 [==============================] - 1s 303ms/step - loss: 0.5761 - binary_accuracy: 0.7415 - auc: 0.8638
Dataset: 9
6/6 [==============================] - 2s 261m

In [ ]:
for acc in avg_acc:
  print(acc)

0.860862135887146
0.9868860840797424
0.9364693760871887
0.9784518480300903
0.9308487772941589
0.9294339418411255
0.849390983581543
0.863771378993988
0.9866358041763306
0.9890824556350708
0.8488849997520447
0.8964110016822815
0.9797243475914001
0.853316068649292
0.9682753086090088
0.9178335070610046
0.9876718521118164
0.9418314695358276
0.9343411922454834
0.9882460832595825
0.956145167350769
0.9306126832962036
0.9886306524276733
0.9891588091850281
0.9835429787635803
0.9899436235427856
0.9729277491569519
0.8700823783874512
0.9925059676170349
0.9023193120956421
0.9837064146995544
0.9836395382881165
0.9828857183456421
0.987529456615448
0.9890211820602417
0.8406006693840027
0.970306396484375
0.9756087064743042
0.9920082688331604
0.9802468419075012
0.9356911778450012
0.9451117515563965
0.9389406442642212
0.8782476186752319
0.991473376750946
0.9310184121131897
0.9361149072647095
0.8571627736091614
0.9870307445526123
0.9847317337989807
0.9862375259399414
0.8955679535865784
0.9268822073936462
0

## Post Fine-Tuning

In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=37:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr])
                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  with open('acc.pkl', 'wb') as file:
    pickle.dump(avg_acc, file)

Dataset: 1
275/275 [==============================] - 4s 8ms/step - loss: 0.4150 - binary_accuracy: 0.8113 - auc: 0.8962
Dataset: 2
558/558 [==============================] - 6s 8ms/step - loss: 0.1202 - binary_accuracy: 0.9587 - auc_1: 0.9903
Dataset: 3
858/858 [==============================] - 8s 8ms/step - loss: 0.1680 - binary_accuracy: 0.9377 - auc_2: 0.9833
Dataset: 4
245/245 [==============================] - 3s 7ms/step - loss: 0.1432 - binary_accuracy: 0.9525 - auc_3: 0.9859
Dataset: 5
386/386 [==============================] - 4s 7ms/step - loss: 0.2844 - binary_accuracy: 0.8810 - auc_4: 0.9518
Dataset: 6
236/236 [==============================] - 3s 7ms/step - loss: 0.2733 - binary_accuracy: 0.8852 - auc_5: 0.9548
Dataset: 7
252/252 [==============================] - 3s 7ms/step - loss: 0.4037 - binary_accuracy: 0.8100 - auc_6: 0.8977
Dataset: 8
278/278 [==============================] - 4s 7ms/step - loss: 0.3975 - binary_accuracy: 0.8166 - auc_7: 0.9007
Dataset: 9
674/674

In [ ]:
for acc in avg_acc:
  print(acc)

0.8961666822433472
0.9903296232223511
0.9833280444145203
0.9858688116073608
0.9518383741378784
0.9547760486602783
0.8977182507514954
0.9006561636924744
0.991176426410675
0.9917778372764587
0.8933342099189758
0.9696537256240845
0.9834957718849182
0.8965144753456116
0.9788564443588257
0.9765400886535645
0.9915405511856079


In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=17:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr,
                                          early_stop])
                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  with open('acc.pkl', 'wb') as file:
    pickle.dump(avg_acc, file)

Dataset: 18
472/472 [==============================] - 6s 9ms/step - loss: 0.1329 - binary_accuracy: 0.9521 - auc: 0.9884
Dataset: 19
505/505 [==============================] - 7s 10ms/step - loss: 0.1962 - binary_accuracy: 0.9210 - auc_1: 0.9768
Dataset: 20
688/688 [==============================] - 7s 9ms/step - loss: 0.1023 - binary_accuracy: 0.9646 - auc_2: 0.9924
Dataset: 21
240/240 [==============================] - 4s 8ms/step - loss: 0.1330 - binary_accuracy: 0.9499 - auc_3: 0.9888
Dataset: 22
276/276 [==============================] - 4s 9ms/step - loss: 0.2585 - binary_accuracy: 0.8949 - auc_4: 0.9605
Dataset: 23
553/553 [==============================] - 6s 9ms/step - loss: 0.1021 - binary_accuracy: 0.9669 - auc_5: 0.9928
Dataset: 24
557/557 [==============================] - 6s 8ms/step - loss: 0.0998 - binary_accuracy: 0.9641 - auc_6: 0.9931
Dataset: 25
582/582 [==============================] - 7s 9ms/step - loss: 0.1383 - binary_accuracy: 0.9548 - auc_7: 0.9871
Dataset: 

In [ ]:
for acc in avg_acc:
  print(acc)

0.9884480237960815
0.9767590165138245
0.9923784136772156
0.9888182282447815
0.9604546427726746
0.9927592873573303
0.993060827255249
0.9870992302894592
0.9935159087181091
0.9796700477600098
0.9091038703918457
0.9939607977867126
0.9328449368476868
0.9875848293304443
0.9897491931915283
0.9882411360740662


In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=37:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr])
                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  with open('acc.pkl', 'wb') as file:
    pickle.dump(avg_acc, file)

Dataset: 38
674/674 [==============================] - 8s 8ms/step - loss: 0.1632 - binary_accuracy: 0.9409 - auc: 0.9811
Dataset: 39
711/711 [==============================] - 8s 8ms/step - loss: 0.0966 - binary_accuracy: 0.9696 - auc_1: 0.9933
Dataset: 40
543/543 [==============================] - 6s 9ms/step - loss: 0.1561 - binary_accuracy: 0.9485 - auc_2: 0.9836
Dataset: 41
541/541 [==============================] - 5s 7ms/step - loss: 0.2109 - binary_accuracy: 0.9213 - auc_3: 0.9748
Dataset: 42
320/320 [==============================] - 4s 9ms/step - loss: 0.1184 - binary_accuracy: 0.9612 - auc_4: 0.9909
Dataset: 43
272/272 [==============================] - 3s 7ms/step - loss: 0.2009 - binary_accuracy: 0.9306 - auc_5: 0.9765
Dataset: 44
348/348 [==============================] - 6s 9ms/step - loss: 0.3453 - binary_accuracy: 0.8539 - auc_6: 0.9307
Dataset: 45
644/644 [==============================] - 7s 9ms/step - loss: 0.0930 - binary_accuracy: 0.9699 - auc_7: 0.9935
Dataset: 4

KeyboardInterrupt: 

In [ ]:
for acc in avg_acc:
  print(acc)

0.9811234474182129
0.9933409094810486
0.9835965633392334
0.9748156070709229
0.9909233450889587
0.9765442609786987
0.930728554725647
0.9935320019721985
0.9573132991790771
0.9581983089447021
0.8984851241111755
0.9915648102760315
0.9893139600753784
0.9896454811096191
0.9394118189811707
0.9902445077896118
0.9941833019256592
0.9751115441322327


In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=55:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr])

                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  # avg_acc.append(data[2])
  print(data[2])

Dataset: 56
270/270 [==============================] - 4s 8ms/step - loss: 0.1340 - binary_accuracy: 0.9541 - auc: 0.9883
0.988250195980072
Dataset: 57
593/593 [==============================] - 8s 9ms/step - loss: 0.1241 - binary_accuracy: 0.9598 - auc_1: 0.9895
0.9894775748252869
Dataset: 58
570/570 [==============================] - 6s 8ms/step - loss: 0.0925 - binary_accuracy: 0.9697 - auc_2: 0.9934
0.9934212565422058
Dataset: 59
350/350 [==============================] - 4s 9ms/step - loss: 0.1102 - binary_accuracy: 0.9635 - auc_3: 0.9918
0.9918239116668701
Dataset: 60
454/454 [==============================] - 5s 8ms/step - loss: 0.1057 - binary_accuracy: 0.9642 - auc_4: 0.9913
0.9912895560264587
Dataset: 61
528/528 [==============================] - 7s 9ms/step - loss: 0.2004 - binary_accuracy: 0.9237 - auc_5: 0.9759
0.975909948348999
Dataset: 62
244/244 [==============================] - 3s 7ms/step - loss: 0.5023 - binary_accuracy: 0.7751 - auc_6: 0.8586
0.8585928678512573
Dat

In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=71:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr])

                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  # avg_acc.append(data[2])
  print(data[2])

Dataset: 72
277/277 [==============================] - 4s 8ms/step - loss: 0.3716 - binary_accuracy: 0.8361 - auc: 0.9147
0.9147188663482666
Dataset: 73
578/578 [==============================] - 6s 8ms/step - loss: 0.1126 - binary_accuracy: 0.9620 - auc_1: 0.9913
0.9913412928581238
Dataset: 74
502/502 [==============================] - 5s 7ms/step - loss: 0.1576 - binary_accuracy: 0.9422 - auc_2: 0.9846
0.9845578074455261
Dataset: 75
678/678 [==============================] - 7s 7ms/step - loss: 0.0892 - binary_accuracy: 0.9696 - auc_3: 0.9943
0.9943222999572754
Dataset: 76
426/426 [==============================] - 5s 8ms/step - loss: 0.2761 - binary_accuracy: 0.8904 - auc_4: 0.9567
0.956654965877533
Dataset: 77
563/563 [==============================] - 5s 6ms/step - loss: 0.1133 - binary_accuracy: 0.9609 - auc_5: 0.9910
0.9910193681716919
Dataset: 78
286/286 [==============================] - 3s 7ms/step - loss: 0.3595 - binary_accuracy: 0.8578 - auc_6: 0.9335
0.933483362197876
Dat

In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=88:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr])

                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  # avg_acc.append(data[2])
  print(data[2])

Dataset: 89
316/316 [==============================] - 4s 7ms/step - loss: 0.3232 - binary_accuracy: 0.8865 - auc: 0.9529
0.9529357552528381
Dataset: 90
396/396 [==============================] - 5s 7ms/step - loss: 0.2433 - binary_accuracy: 0.9010 - auc_1: 0.9657
0.9657166600227356
Dataset: 91
347/347 [==============================] - 4s 7ms/step - loss: 0.3767 - binary_accuracy: 0.8345 - auc_2: 0.9163
0.9163068532943726
Dataset: 92
662/662 [==============================] - 6s 7ms/step - loss: 0.1960 - binary_accuracy: 0.9307 - auc_3: 0.9777
0.9777268171310425
Dataset: 93
533/533 [==============================] - 5s 7ms/step - loss: 0.0723 - binary_accuracy: 0.9783 - auc_4: 0.9948
0.994799017906189
Dataset: 94
1048/1048 [==============================] - 9s 7ms/step - loss: 0.1983 - binary_accuracy: 0.9289 - auc_5: 0.9785
0.9785394072532654
Dataset: 95
249/249 [==============================] - 3s 7ms/step - loss: 0.2603 - binary_accuracy: 0.8918 - auc_6: 0.9597
0.9596908688545227


In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=107:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr])

                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  # avg_acc.append(data[2])
  print(data[2])

Dataset: 108
675/675 [==============================] - 7s 7ms/step - loss: 0.1165 - binary_accuracy: 0.9588 - auc: 0.9908
0.9908350110054016
Dataset: 109
245/245 [==============================] - 4s 7ms/step - loss: 0.2002 - binary_accuracy: 0.9266 - auc_1: 0.9763
0.9762880802154541
Dataset: 110
655/655 [==============================] - 6s 7ms/step - loss: 0.0954 - binary_accuracy: 0.9675 - auc_2: 0.9932
0.9931941628456116
Dataset: 111
357/357 [==============================] - 4s 7ms/step - loss: 0.2078 - binary_accuracy: 0.9227 - auc_3: 0.9747
0.9747368693351746
Dataset: 112
245/245 [==============================] - 3s 7ms/step - loss: 0.4703 - binary_accuracy: 0.7803 - auc_4: 0.8689
0.8688626885414124
Dataset: 113
757/757 [==============================] - 7s 7ms/step - loss: 0.0998 - binary_accuracy: 0.9705 - auc_5: 0.9923
0.9923193454742432
Dataset: 114
505/505 [==============================] - 5s 6ms/step - loss: 0.0875 - binary_accuracy: 0.9703 - auc_6: 0.9941
0.99406993389

In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=125:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr])

                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  # avg_acc.append(data[2])
  print(data[2])

Dataset: 126
340/340 [==============================] - 4s 7ms/step - loss: 0.1909 - binary_accuracy: 0.9269 - auc: 0.9779
0.9778608679771423
Dataset: 127
711/711 [==============================] - 6s 7ms/step - loss: 0.1248 - binary_accuracy: 0.9573 - auc_1: 0.9890
0.9890339374542236
Dataset: 128
494/494 [==============================] - 5s 7ms/step - loss: 0.0682 - binary_accuracy: 0.9785 - auc_2: 0.9957
0.9956973195075989
Dataset: 129
350/350 [==============================] - 4s 7ms/step - loss: 0.2151 - binary_accuracy: 0.9190 - auc_3: 0.9723
0.9722987413406372
Dataset: 130
317/317 [==============================] - 3s 7ms/step - loss: 0.2670 - binary_accuracy: 0.8923 - auc_4: 0.9594
0.959394633769989
Dataset: 131
452/452 [==============================] - 5s 7ms/step - loss: 0.1966 - binary_accuracy: 0.9256 - auc_5: 0.9770
0.9770217537879944
Dataset: 132
671/671 [==============================] - 6s 8ms/step - loss: 0.1055 - binary_accuracy: 0.9657 - auc_6: 0.9924
0.992427885532

In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=138:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr])

                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  # avg_acc.append(data[2])
  print(data[2])

Dataset: 139
361/361 [==============================] - 4s 8ms/step - loss: 0.1678 - binary_accuracy: 0.9368 - auc: 0.9827
0.9826581478118896
Dataset: 140
432/432 [==============================] - 6s 10ms/step - loss: 0.2621 - binary_accuracy: 0.8927 - auc_1: 0.9602
0.9601830840110779
Dataset: 141
267/267 [==============================] - 6s 15ms/step - loss: 0.2261 - binary_accuracy: 0.9094 - auc_2: 0.9699
0.9699382185935974
Dataset: 142
557/557 [==============================] - 7s 10ms/step - loss: 0.1491 - binary_accuracy: 0.9483 - auc_3: 0.9851
0.9851206541061401
Dataset: 143
653/653 [==============================] - 6s 8ms/step - loss: 0.1723 - binary_accuracy: 0.9452 - auc_4: 0.9834
0.983420193195343
Dataset: 144
292/292 [==============================] - 4s 8ms/step - loss: 0.2904 - binary_accuracy: 0.8867 - auc_5: 0.9557
0.9557333588600159
Dataset: 145
497/497 [==============================] - 5s 8ms/step - loss: 0.0614 - binary_accuracy: 0.9838 - auc_6: 0.9956
0.995593428

KeyboardInterrupt: 

In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=149:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr])

                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  # avg_acc.append(data[2])
  print(data[2])

Dataset: 150
525/525 [==============================] - 6s 8ms/step - loss: 0.1498 - binary_accuracy: 0.9430 - auc: 0.9858
0.9858456254005432
Dataset: 151
283/283 [==============================] - 4s 9ms/step - loss: 0.0646 - binary_accuracy: 0.9813 - auc_1: 0.9955
0.995506227016449
Dataset: 152
556/556 [==============================] - 7s 9ms/step - loss: 0.1032 - binary_accuracy: 0.9668 - auc_2: 0.9930
0.9929628968238831
Dataset: 153
264/264 [==============================] - 3s 7ms/step - loss: 0.3564 - binary_accuracy: 0.8453 - auc_3: 0.9240
0.9239739179611206
Dataset: 154
431/431 [==============================] - 4s 7ms/step - loss: 0.1297 - binary_accuracy: 0.9547 - auc_4: 0.9875
0.9874840974807739
Dataset: 155
586/586 [==============================] - 6s 7ms/step - loss: 0.2244 - binary_accuracy: 0.9238 - auc_5: 0.9722
0.9721807837486267
Dataset: 156
279/279 [==============================] - 5s 10ms/step - loss: 0.3067 - binary_accuracy: 0.8748 - auc_6: 0.9460
0.94598156213

In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=168:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr])

                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  # avg_acc.append(data[2])
  print(data[2])

Dataset: 169
245/245 [==============================] - 3s 7ms/step - loss: 0.3975 - binary_accuracy: 0.8190 - auc: 0.9014
0.9013733267784119
Dataset: 170
679/679 [==============================] - 7s 7ms/step - loss: 0.0859 - binary_accuracy: 0.9719 - auc_1: 0.9944
0.9943748116493225
Dataset: 171
380/380 [==============================] - 4s 7ms/step - loss: 0.2273 - binary_accuracy: 0.9058 - auc_2: 0.9665
0.966536283493042
Dataset: 172
289/289 [==============================] - 3s 8ms/step - loss: 0.3791 - binary_accuracy: 0.8336 - auc_3: 0.9140
0.9139936566352844
Dataset: 173
342/342 [==============================] - 4s 9ms/step - loss: 0.2560 - binary_accuracy: 0.8949 - auc_4: 0.9603
0.9603320360183716
Dataset: 174
915/915 [==============================] - 8s 7ms/step - loss: 0.1405 - binary_accuracy: 0.9495 - auc_5: 0.9873
0.9872773885726929
Dataset: 175


In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=149:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr])

                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  # avg_acc.append(data[2])
  print(data[2])

Dataset: 175
532/532 [==============================] - 5s 7ms/step - loss: 0.0992 - binary_accuracy: 0.9678 - auc: 0.9924
0.9924378991127014
Dataset: 176
568/568 [==============================] - 6s 7ms/step - loss: 0.1136 - binary_accuracy: 0.9617 - auc_1: 0.9909
0.9908649325370789
Dataset: 177
480/480 [==============================] - 5s 8ms/step - loss: 0.1747 - binary_accuracy: 0.9316 - auc_2: 0.9805
0.9805411100387573
Dataset: 178
513/513 [==============================] - 5s 7ms/step - loss: 0.1076 - binary_accuracy: 0.9632 - auc_3: 0.9920
0.991970956325531
Dataset: 179
341/341 [==============================] - 5s 8ms/step - loss: 0.3313 - binary_accuracy: 0.8567 - auc_4: 0.9349
0.9349426627159119
Dataset: 180
251/251 [==============================] - 3s 7ms/step - loss: 0.2745 - binary_accuracy: 0.8883 - auc_5: 0.9567
0.9567063450813293
Dataset: 181
485/485 [==============================] - 5s 6ms/step - loss: 0.0842 - binary_accuracy: 0.9711 - auc_6: 0.9946
0.994585454463

In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=193:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr])

                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  # avg_acc.append(data[2])
  print(data[2])

Dataset: 194
461/461 [==============================] - 5s 7ms/step - loss: 0.0762 - binary_accuracy: 0.9759 - auc: 0.9953
0.9952648878097534
Dataset: 195
313/313 [==============================] - 4s 9ms/step - loss: 0.2857 - binary_accuracy: 0.8899 - auc_1: 0.9559
0.9559028744697571
Dataset: 196
288/288 [==============================] - 3s 7ms/step - loss: 0.1017 - binary_accuracy: 0.9690 - auc_2: 0.9914
0.9914003610610962
Dataset: 197
569/569 [==============================] - 6s 8ms/step - loss: 0.0908 - binary_accuracy: 0.9700 - auc_3: 0.9935
0.9935169816017151
Dataset: 198
239/239 [==============================] - 3s 7ms/step - loss: 0.3121 - binary_accuracy: 0.8733 - auc_4: 0.9446
0.9446194171905518
Dataset: 199
534/534 [==============================] - 5s 7ms/step - loss: 0.1141 - binary_accuracy: 0.9620 - auc_5: 0.9904
0.990407407283783
Dataset: 200
243/243 [==============================] - 3s 7ms/step - loss: 0.3857 - binary_accuracy: 0.8273 - auc_6: 0.9101
0.910066783428

In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=212:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr])

                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  # avg_acc.append(data[2])
  print(data[2])

Dataset: 213
535/535 [==============================] - 6s 7ms/step - loss: 0.1065 - binary_accuracy: 0.9651 - auc: 0.9923
0.9923228025436401
Dataset: 214
615/615 [==============================] - 6s 7ms/step - loss: 0.1498 - binary_accuracy: 0.9500 - auc_1: 0.9854
0.9853849411010742
Dataset: 215
461/461 [==============================] - 5s 8ms/step - loss: 0.1038 - binary_accuracy: 0.9653 - auc_2: 0.9921
0.9921091794967651
Dataset: 216
560/560 [==============================] - 6s 8ms/step - loss: 0.1007 - binary_accuracy: 0.9662 - auc_3: 0.9926
0.9925958514213562
Dataset: 217
494/494 [==============================] - 7s 11ms/step - loss: 0.1322 - binary_accuracy: 0.9532 - auc_4: 0.9890
0.9890058636665344
Dataset: 218
456/456 [==============================] - 5s 8ms/step - loss: 0.3583 - binary_accuracy: 0.8545 - auc_5: 0.9300
0.9299664497375488
Dataset: 219
341/341 [==============================] - 4s 7ms/step - loss: 0.3847 - binary_accuracy: 0.8321 - auc_6: 0.9121
0.9121113419

In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i<=230:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=30,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr])

                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  # avg_acc.append(data[2])
  print(data[2])

Dataset: 231
553/553 [==============================] - 6s 8ms/step - loss: 0.1412 - binary_accuracy: 0.9547 - auc: 0.9876
0.9875994920730591
Dataset: 232
262/262 [==============================] - 4s 7ms/step - loss: 0.3674 - binary_accuracy: 0.8348 - auc_1: 0.9185
0.9184787273406982
Dataset: 233
352/352 [==============================] - 4s 7ms/step - loss: 0.3059 - binary_accuracy: 0.8708 - auc_2: 0.9452
0.9452080726623535
Dataset: 234
992/992 [==============================] - 8s 7ms/step - loss: 0.1380 - binary_accuracy: 0.9538 - auc_3: 0.9879
0.987942099571228
Dataset: 235
258/258 [==============================] - 4s 9ms/step - loss: 0.3523 - binary_accuracy: 0.8525 - auc_4: 0.9294
0.9294230341911316
Dataset: 236
595/595 [==============================] - 6s 8ms/step - loss: 0.1137 - binary_accuracy: 0.9657 - auc_5: 0.9914
0.991375207901001
Dataset: 237
385/385 [==============================] - 5s 7ms/step - loss: 0.1733 - binary_accuracy: 0.9372 - auc_6: 0.9813
0.9812868833541

In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
indices=[1,7,11,14,48,62,88,99,105,112,124,147,183,193,203,223,229]
for train_label,test_feature,test_label in zip(train_labels,test_features,test_labels):
  i=i+1
  if i not in indices:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_large_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(large_train_features[i-1],dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=1024,
                              epochs=2,
                              verbose=0,
                              validation_split=0.01,
                              validation_batch_size=4096,
                              callbacks=([early_stop])

                                          )
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  print(data[2])

Dataset: 1
275/275 [==============================] - 4s 8ms/step - loss: 0.4022 - binary_accuracy: 0.8163 - auc_2: 0.8986
0.8985990881919861
Dataset: 7
252/252 [==============================] - 4s 9ms/step - loss: 0.4013 - binary_accuracy: 0.8057 - auc_3: 0.8984
0.8984169960021973
Dataset: 11
246/246 [==============================] - 3s 8ms/step - loss: 0.4108 - binary_accuracy: 0.8093 - auc_4: 0.8963
0.8962745666503906
Dataset: 14
236/236 [==============================] - 4s 10ms/step - loss: 0.4300 - binary_accuracy: 0.8030 - auc_5: 0.8975
0.8975381851196289
Dataset: 48
267/267 [==============================] - 3s 8ms/step - loss: 0.4039 - binary_accuracy: 0.8105 - auc_6: 0.9004
0.9004409313201904
Dataset: 62
244/244 [==============================] - 3s 8ms/step - loss: 0.4626 - binary_accuracy: 0.7783 - auc_7: 0.8660
0.8659942150115967
Dataset: 88
250/250 [==============================] - 4s 11ms/step - loss: 0.4421 - binary_accuracy: 0.7847 - auc_8: 0.8815
0.8814924955368042

# Medium Datasets

## Pre Fine Tuning

In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for test_feature,test_label in zip(test_features,test_labels):
  i=i+1
  print("Dataset:",i)
  data = CBLANE.evaluate(test_feature,test_label,verbose=1,batch_size=4096)
  avg_acc.append(data[2])
print(np.mean(avg_acc))

Dataset: 1
2/2 [==============================] - 8s 1s/step - loss: 0.3474 - binary_accuracy: 0.8451 - auc_1: 0.9248
Dataset: 2
2/2 [==============================] - 1s 1s/step - loss: 0.3741 - binary_accuracy: 0.8292 - auc_1: 0.9158
Dataset: 3
2/2 [==============================] - 1s 789ms/step - loss: 0.2989 - binary_accuracy: 0.8672 - auc_1: 0.9458
Dataset: 4
1/1 [==============================] - 1s 1s/step - loss: 0.2041 - binary_accuracy: 0.9211 - auc_1: 0.9825
Dataset: 5
1/1 [==============================] - 1s 1s/step - loss: 0.2677 - binary_accuracy: 0.8890 - auc_1: 0.9562
Dataset: 6
2/2 [==============================] - 1s 1s/step - loss: 0.3065 - binary_accuracy: 0.8645 - auc_1: 0.9419
Dataset: 7
1/1 [==============================] - 1s 1s/step - loss: 0.3587 - binary_accuracy: 0.8456 - auc_1: 0.9203
Dataset: 8
2/2 [==============================] - 2s 2s/step - loss: 0.3820 - binary_accuracy: 0.8287 - auc_1: 0.9111
Dataset: 9
1/1 [==============================] - 1s 

## Fine-Tuning

In [ ]:
#pre fine-tuning
CBLANE.evaluate(medium_test_features,medium_test_labels,batch_size=4096)

359/359 [==============================] - 72s 195ms/step - loss: 0.3401 - binary_accuracy: 0.8500 - auc_1: 0.9290


[0.3400660753250122, 0.8500096797943115, 0.9290493726730347]

In [ ]:
from keras.models import load_model
CBLANE = load_model("CBLANE.keras")
CBLANE.compile(loss='binary_crossentropy',
                             optimizer=Adam(learning_rate=0.0001),
                             metrics=[BinaryAccuracy(),
                                      # Precision(),
                                      # Recall(),
                                      AUC(),
                                      # SensitivityAtSpecificity(0.5),
                                      # SpecificityAtSensitivity(0.5),
                                      ]
                             )
history = CBLANE.fit(tf.constant(medium_train_features,dtype=tf.bool),
                             tf.constant(medium_train_labels,dtype=tf.bool),
                             batch_size=1024,
                             epochs=30,
                             verbose=1,
                             validation_split=0.1,
                             validation_batch_size=4096,
                             callbacks=([reduce_lr,
                                         SaveSubModels()]))

Epoch 1/30
   6/5146 [..............................] - ETA: 13:15 - loss: 0.4291 - binary_accuracy: 0.7987 - auc: 0.8830

5146/5146 [==============================] - 909s 173ms/step - loss: 0.3819 - binary_accuracy: 0.8255 - auc: 0.9080 - val_loss: 0.3548 - val_binary_accuracy: 0.8404 - val_auc: 0.9210 - lr: 1.0000e-04
Epoch 2/30
5146/5146 [==============================] - 886s 172ms/step - loss: 0.3671 - binary_accuracy: 0.8335 - auc: 0.9154 - val_loss: 0.3475 - val_binary_accuracy: 0.8439 - val_auc: 0.9245 - lr: 1.0000e-04
Epoch 3/30
5146/5146 [==============================] - 876s 170ms/step - loss: 0.3589 - binary_accuracy: 0.8380 - auc: 0.9194 - val_loss: 0.3431 - val_binary_accuracy: 0.8464 - val_auc: 0.9271 - lr: 1.0000e-04
Epoch 4/30
5146/5146 [==============================] - 888s 173ms/step - loss: 0.3525 - binary_accuracy: 0.8414 - auc: 0.9224 - val_loss: 0.3371 - val_binary_accuracy: 0.8497 - val_auc: 0.9291 - lr: 1.0000e-04
Epoch 5/30
5146/5146 [==============================] - 886s 172ms/step - loss: 0.3473 - binary_accuracy: 0.8446 - auc: 0.9249 - val_loss: 0.3337 - val_binary_accuracy

## Post Fine-Tuning

In [ ]:
from keras.models import load_model
i = 0
avg_acc = []
for train_feature,train_label,test_feature,test_label in zip(train_features,train_labels,test_features,test_labels):
  i=i+1
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_medium_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.0001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(train_feature,dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=4096,
                              epochs=30,
                              verbose=0,
                              validation_split=0.1,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr,
                                          early_stop]))
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  if i%10==0:
    with open('acc.pkl', 'wb') as file:
      pickle.dump(avg_acc, file)
    break

Dataset: 1
187/187 [==============================] - 5s 13ms/step - loss: 0.3335 - binary_accuracy: 0.8556 - auc: 0.9312
Dataset: 2
205/205 [==============================] - 5s 13ms/step - loss: 0.3560 - binary_accuracy: 0.8385 - auc_1: 0.9212
Dataset: 3
173/173 [==============================] - 4s 12ms/step - loss: 0.2800 - binary_accuracy: 0.8806 - auc_2: 0.9550
Dataset: 4
94/94 [==============================] - 2s 10ms/step - loss: 0.1806 - binary_accuracy: 0.9285 - auc_3: 0.9866
Dataset: 5
69/69 [==============================] - 2s 8ms/step - loss: 0.2634 - binary_accuracy: 0.8885 - auc_4: 0.9573
Dataset: 6
191/191 [==============================] - 4s 13ms/step - loss: 0.3010 - binary_accuracy: 0.8714 - auc_5: 0.9442
Dataset: 7
95/95 [==============================] - 2s 10ms/step - loss: 0.3547 - binary_accuracy: 0.8459 - auc_6: 0.9217
Dataset: 8
233/233 [==============================] - 3s 9ms/step - loss: 0.3708 - binary_accuracy: 0.8338 - auc_7: 0.9148
Dataset: 9
82/82 [

In [ ]:
for acc in avg_acc:
  print(acc)

0.9312074780464172
0.921187162399292
0.9549838900566101
0.9865534901618958
0.9573073387145996
0.9442133903503418
0.9217252731323242
0.9148148894309998
0.8716135621070862
0.9714758992195129


In [ ]:
import gc
from keras.models import load_model
i = 0
avg_acc = []
for train_feature,train_label,test_feature,test_label in zip(train_features,train_labels,test_features,test_labels):
  i=i+1
  if i<=10:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_medium_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.0001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(train_feature,dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=4096,
                              epochs=30,
                              verbose=0,
                              validation_split=0.1,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr,
                                          early_stop]))
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  if i%10==0:
    with open('acc.pkl', 'wb') as file:
      pickle.dump(avg_acc, file)
    gc.collect()

Dataset: 11
181/181 [==============================] - 3s 8ms/step - loss: 0.3374 - binary_accuracy: 0.8502 - auc: 0.9307
Dataset: 12
55/55 [==============================] - 3s 12ms/step - loss: 0.3401 - binary_accuracy: 0.8513 - auc_1: 0.9297
Dataset: 13
150/150 [==============================] - 4s 12ms/step - loss: 0.2815 - binary_accuracy: 0.8810 - auc_2: 0.9517
Dataset: 14
66/66 [==============================] - 2s 11ms/step - loss: 0.4335 - binary_accuracy: 0.8049 - auc_3: 0.8943
Dataset: 15
223/223 [==============================] - 4s 9ms/step - loss: 0.2414 - binary_accuracy: 0.9057 - auc_4: 0.9662
Dataset: 16
59/59 [==============================] - 2s 8ms/step - loss: 0.3727 - binary_accuracy: 0.8433 - auc_5: 0.9212
Dataset: 17
172/172 [==============================] - 4s 9ms/step - loss: 0.2907 - binary_accuracy: 0.8777 - auc_6: 0.9484
Dataset: 18
168/168 [==============================] - 3s 9ms/step - loss: 0.3495 - binary_accuracy: 0.8473 - auc_7: 0.9275
Dataset: 19
4

In [ ]:
for acc in avg_acc:
  print(acc)

0.9306551814079285
0.9296557307243347
0.9516918063163757
0.894320011138916
0.9662401080131531
0.9212194085121155
0.9484221339225769
0.9274718761444092
0.9428207874298096
0.961245059967041
0.9393441677093506
0.907924234867096
0.9321599006652832
0.928375244140625
0.9502923488616943
0.9573821425437927
0.9591288566589355
0.9674414992332458
0.9632229208946228
0.8600559830665588
0.9087160229682922
0.8309941291809082
0.8967456221580505
0.9485831260681152
0.9441002607345581
0.9290468096733093
0.9025838971138
0.9389432072639465
0.8458400368690491
0.8819204568862915
0.9003790020942688
0.9512417316436768
0.9200512766838074
0.9509711861610413
0.9557961821556091
0.9653223752975464
0.8547034859657288
0.9525251984596252
0.9397828578948975
0.9211435317993164


In [ ]:
import gc
from keras.models import load_model
i = 0
avg_acc = []
for train_feature,train_label,test_feature,test_label in zip(train_features,train_labels,test_features,test_labels):
  i=i+1
  if i<=50:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_medium_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.0001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(train_feature,dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=4096,
                              epochs=30,
                              verbose=0,
                              validation_split=0.1,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr,
                                          early_stop]))
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  if i%10==0:
    with open('acc.pkl', 'wb') as file:
      pickle.dump(avg_acc, file)
    gc.collect()

Dataset: 51
41/41 [==============================] - 2s 13ms/step - loss: 0.3035 - binary_accuracy: 0.8631 - auc: 0.9434
Dataset: 52
216/216 [==============================] - 3s 8ms/step - loss: 0.3144 - binary_accuracy: 0.8666 - auc_1: 0.9407
Dataset: 53
107/107 [==============================] - 3s 10ms/step - loss: 0.4609 - binary_accuracy: 0.7884 - auc_2: 0.8709
Dataset: 54
55/55 [==============================] - 2s 8ms/step - loss: 0.2094 - binary_accuracy: 0.9138 - auc_3: 0.9731
Dataset: 55
54/54 [==============================] - 2s 13ms/step - loss: 0.2771 - binary_accuracy: 0.8820 - auc_4: 0.9533
Dataset: 56
126/126 [==============================] - 2s 9ms/step - loss: 0.2121 - binary_accuracy: 0.9181 - auc_5: 0.9792
Dataset: 57
215/215 [==============================] - 3s 8ms/step - loss: 0.3380 - binary_accuracy: 0.8500 - auc_6: 0.9304
Dataset: 58
31/31 [==============================] - 2s 11ms/step - loss: 0.4193 - binary_accuracy: 0.8002 - auc_7: 0.8873
Dataset: 59
14

In [ ]:
for acc in avg_acc:
  print(acc)

0.9434425830841064
0.9407087564468384
0.8709458708763123
0.9731247425079346
0.9532525539398193
0.9792302250862122
0.9304027557373047
0.8872883915901184
0.8798696994781494
0.940461277961731
0.9152224659919739
0.8850703835487366
0.8894384503364563
0.9584742784500122
0.9201711416244507
0.9596551656723022
0.9373490810394287
0.9325723648071289
0.9414169788360596
0.9222031235694885
0.8896247744560242
0.9276741743087769
0.9564241170883179
0.970929741859436
0.9840898513793945
0.9068130850791931
0.9538402557373047
0.900589644908905
0.8918517231941223
0.955828845500946
0.9283287525177002
0.9561659097671509
0.9270933866500854
0.934490978717804
0.9228633642196655
0.9510050415992737
0.956079363822937
0.9196365475654602
0.9782773852348328
0.9447871446609497


In [ ]:
import gc
from keras.models import load_model
i = 0
avg_acc = []
for train_feature,train_label,test_feature,test_label in zip(train_features,train_labels,test_features,test_labels):
  i=i+1
  if i<=90:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_medium_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.0001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(train_feature,dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=4096,
                              epochs=30,
                              verbose=0,
                              validation_split=0.1,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr,
                                          early_stop]))
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  with open('acc.pkl', 'wb') as file:
    pickle.dump(avg_acc, file)
  gc.collect()

Dataset: 91
37/37 [==============================] - 2s 13ms/step - loss: 0.2810 - binary_accuracy: 0.8821 - auc: 0.9521
Dataset: 92
157/157 [==============================] - 3s 9ms/step - loss: 0.2151 - binary_accuracy: 0.9142 - auc_1: 0.9795
Dataset: 93
85/85 [==============================] - 2s 9ms/step - loss: 0.2749 - binary_accuracy: 0.8815 - auc_2: 0.9536
Dataset: 94
232/232 [==============================] - 3s 8ms/step - loss: 0.3856 - binary_accuracy: 0.8291 - auc_3: 0.9077
Dataset: 95
224/224 [==============================] - 3s 7ms/step - loss: 0.2897 - binary_accuracy: 0.8793 - auc_4: 0.9492
Dataset: 96
136/136 [==============================] - 3s 9ms/step - loss: 0.3499 - binary_accuracy: 0.8452 - auc_5: 0.9292
Dataset: 97
200/200 [==============================] - 3s 7ms/step - loss: 0.3017 - binary_accuracy: 0.8722 - auc_6: 0.9440
Dataset: 98
74/74 [==============================] - 2s 10ms/step - loss: 0.3571 - binary_accuracy: 0.8367 - auc_7: 0.9216
Dataset: 99
72

In [ ]:
for acc in avg_acc:
  print(acc)

0.952144980430603
0.9794921875
0.9536312222480774
0.9077019691467285
0.9491674900054932
0.9291517734527588
0.9439911842346191
0.921553909778595
0.9596505165100098
0.981364369392395
0.876914918422699
0.9620929956436157
0.9263676404953003
0.9202947020530701
0.9526789784431458
0.9349732995033264
0.8937196135520935
0.9485183954238892
0.9661709070205688
0.9507594704627991
0.8821559548377991
0.9669102430343628
0.8957704305648804
0.9181241989135742
0.9371542930603027
0.9275383353233337
0.860419750213623
0.9141083359718323
0.8843063116073608
0.9590222239494324
0.906002938747406
0.9235966205596924
0.945484459400177
0.9417846202850342
0.9752907752990723
0.9426071047782898
0.9380014538764954
0.8750603199005127
0.9296598434448242
0.7945277690887451
0.9615758061408997
0.9544819593429565
0.9258757829666138
0.8871482610702515
0.9312261939048767
0.9349220395088196
0.8834552764892578


In [ ]:
import gc
from keras.models import load_model
i = 0
avg_acc = []
for train_feature,train_label,test_feature,test_label in zip(train_features,train_labels,test_features,test_labels):
  i=i+1
  if i<=137:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_medium_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.0001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(train_feature,dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=4096,
                              epochs=30,
                              verbose=0,
                              validation_split=0.1,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr,
                                          early_stop]))
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  with open('acc.pkl', 'wb') as file:
    pickle.dump(avg_acc, file)
  gc.collect()

Dataset: 138
165/165 [==============================] - 3s 8ms/step - loss: 0.3558 - binary_accuracy: 0.8422 - auc: 0.9243
Dataset: 139
131/131 [==============================] - 3s 9ms/step - loss: 0.3331 - binary_accuracy: 0.8554 - auc_1: 0.9326
Dataset: 140
193/193 [==============================] - 4s 12ms/step - loss: 0.4199 - binary_accuracy: 0.8070 - auc_2: 0.8963
Dataset: 141
108/108 [==============================] - 3s 12ms/step - loss: 0.3146 - binary_accuracy: 0.8643 - auc_3: 0.9399
Dataset: 142
36/36 [==============================] - 2s 14ms/step - loss: 0.2330 - binary_accuracy: 0.9085 - auc_4: 0.9679
Dataset: 143
170/170 [==============================] - 3s 7ms/step - loss: 0.3422 - binary_accuracy: 0.8450 - auc_5: 0.9315
Dataset: 144
230/230 [==============================] - 3s 9ms/step - loss: 0.3263 - binary_accuracy: 0.8579 - auc_6: 0.9349
Dataset: 145
175/175 [==============================] - 3s 9ms/step - loss: 0.2397 - binary_accuracy: 0.8992 - auc_7: 0.9649
D

In [ ]:
for acc in avg_acc:
  print(acc)

0.9243074059486389
0.9325570464134216
0.8963478207588196
0.9398764967918396
0.9678941965103149
0.9315146207809448
0.9349467158317566
0.9649478197097778
0.8560346364974976
0.9137784838676453
0.9342442750930786
0.816816508769989
0.957145094871521
0.881974458694458
0.8638526201248169
0.9115430116653442
0.9777678847312927
0.9303597807884216
0.9226472973823547
0.9419198036193848
0.9565771222114563
0.9118928909301758
0.9441162943840027
0.9456779956817627
0.9687814116477966
0.9398624897003174
0.9714159369468689
0.9661412239074707
0.8615740537643433
0.9387581944465637
0.8981323838233948
0.942013680934906
0.9463198781013489
0.9180708527565002
0.9289506673812866
0.8273903727531433
0.967467725276947
0.9582687616348267
0.9462282657623291
0.9318883419036865
0.966103732585907
0.9747071862220764
0.9383089542388916
0.9382795691490173
0.9514319896697998
0.9458338022232056


In [ ]:
import gc
from keras.models import load_model
i = 0
avg_acc = []
for train_feature,train_label,test_feature,test_label in zip(train_features,train_labels,test_features,test_labels):
  i=i+1
  if i<=183:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_medium_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.0001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(train_feature,dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=4096,
                              epochs=30,
                              verbose=0,
                              validation_split=0.1,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr,
                                          early_stop]))
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  with open('acc.pkl', 'wb') as file:
    pickle.dump(avg_acc, file)
  gc.collect()

Dataset: 184
130/130 [==============================] - 3s 9ms/step - loss: 0.4320 - binary_accuracy: 0.8026 - auc: 0.8847
Dataset: 185
38/38 [==============================] - 2s 14ms/step - loss: 0.2946 - binary_accuracy: 0.8748 - auc_1: 0.9471
Dataset: 186
146/146 [==============================] - 3s 12ms/step - loss: 0.3262 - binary_accuracy: 0.8543 - auc_2: 0.9353
Dataset: 187
222/222 [==============================] - 4s 9ms/step - loss: 0.2284 - binary_accuracy: 0.9052 - auc_3: 0.9683
Dataset: 188
221/221 [==============================] - 3s 8ms/step - loss: 0.3692 - binary_accuracy: 0.8377 - auc_4: 0.9202
Dataset: 189
101/101 [==============================] - 2s 9ms/step - loss: 0.2903 - binary_accuracy: 0.8785 - auc_5: 0.9488
Dataset: 190
62/62 [==============================] - 2s 11ms/step - loss: 0.2382 - binary_accuracy: 0.8993 - auc_6: 0.9662
Dataset: 191
29/29 [==============================] - 2s 13ms/step - loss: 0.2592 - binary_accuracy: 0.8866 - auc_7: 0.9616
Data

In [ ]:
import pickle
with open('acc.pkl', 'rb') as file:
  avg_acc = pickle.load(file)
for acc in avg_acc:
  print(acc)

0.8846648931503296
0.9471480846405029
0.9352574348449707
0.9683359265327454
0.920158326625824
0.9488080739974976
0.9661758542060852
0.961597740650177
0.9267008304595947
0.9363128542900085
0.9219815731048584
0.9535526037216187
0.9221689105033875
0.9781924486160278
0.9798941612243652
0.9190000295639038
0.9650973081588745
0.8669565916061401
0.9399836659431458
0.9265304803848267
0.8781782388687134
0.9659824967384338
0.9668212532997131
0.9308528900146484
0.8783012628555298
0.9172869920730591
0.91311115026474
0.9601110219955444
0.962104320526123
0.957688570022583
0.9353210926055908
0.969108521938324
0.929503858089447
0.8320202231407166
0.9460240602493286
0.952167809009552
0.9828872680664062
0.8864848017692566
0.8529946804046631
0.8832821249961853
0.9712824821472168
0.9442206025123596
0.9343276619911194
0.8906334638595581
0.9671363830566406
0.7485958337783813
0.8402615189552307


In [ ]:
import gc
from keras.models import load_model
i = 0
avg_acc = []
for train_feature,train_label,test_feature,test_label in zip(train_features,train_labels,test_features,test_labels):
  i=i+1
  if i<230:
    continue
  if i==241:
    break
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_medium_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.0001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(train_feature,dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=4096,
                              epochs=30,
                              verbose=0,
                              validation_split=0.1,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr,
                                          early_stop]))
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  with open('acc.pkl', 'wb') as file:
    pickle.dump(avg_acc, file)
  gc.collect()

Dataset: 230
25/25 [==============================] - 2s 15ms/step - loss: 0.5152 - binary_accuracy: 0.7617 - auc: 0.8405
Dataset: 231
51/51 [==============================] - 2s 12ms/step - loss: 0.3436 - binary_accuracy: 0.8411 - auc_1: 0.9277
Dataset: 232
200/200 [==============================] - 3s 8ms/step - loss: 0.3487 - binary_accuracy: 0.8466 - auc_2: 0.9251
Dataset: 233
116/116 [==============================] - 3s 13ms/step - loss: 0.5305 - binary_accuracy: 0.7555 - auc_3: 0.8386
Dataset: 234
67/67 [==============================] - 3s 10ms/step - loss: 0.5884 - binary_accuracy: 0.7108 - auc_4: 0.8055
Dataset: 235
197/197 [==============================] - 3s 9ms/step - loss: 0.3478 - binary_accuracy: 0.8439 - auc_5: 0.9286
Dataset: 236
64/64 [==============================] - 2s 11ms/step - loss: 0.2765 - binary_accuracy: 0.8755 - auc_6: 0.9538
Dataset: 237
170/170 [==============================] - 3s 8ms/step - loss: 0.3603 - binary_accuracy: 0.8418 - auc_7: 0.9210
Datas

In [ ]:
# import pickle
# with open('acc.pkl', 'rb') as file:
#   avg_acc = pickle.load(file)
for acc in avg_acc:
  print(acc)

0.8405270576477051
0.927676260471344
0.9250666499137878
0.8385700583457947
0.8054885268211365
0.9286465048789978
0.9538219571113586
0.9210497736930847
0.9301666021347046
0.9589602947235107
0.8596146702766418


In [ ]:
import gc
from keras.models import load_model
i = 0
avg_acc = []
for train_feature,train_label,test_feature,test_label in zip(train_features,train_labels,test_features,test_labels):
  i=i+1
  if i<=240:
    continue
  if i>=290:
    break
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_medium_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.0001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(train_feature,dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=4096,
                              epochs=30,
                              verbose=0,
                              validation_split=0.1,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr,
                                          early_stop]))
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  with open('acc.pkl', 'wb') as file:
    pickle.dump(avg_acc, file)
  gc.collect()

Dataset: 241
80/80 [==============================] - 3s 9ms/step - loss: 0.2215 - binary_accuracy: 0.9091 - auc: 0.9698
Dataset: 242
218/218 [==============================] - 4s 9ms/step - loss: 0.3404 - binary_accuracy: 0.8517 - auc_1: 0.9300
Dataset: 243
82/82 [==============================] - 2s 9ms/step - loss: 0.1987 - binary_accuracy: 0.9173 - auc_2: 0.9767
Dataset: 244
227/227 [==============================] - 3s 8ms/step - loss: 0.2041 - binary_accuracy: 0.9200 - auc_3: 0.9764
Dataset: 245
199/199 [==============================] - 3s 8ms/step - loss: 0.3348 - binary_accuracy: 0.8506 - auc_4: 0.9333
Dataset: 246
218/218 [==============================] - 3s 7ms/step - loss: 0.3085 - binary_accuracy: 0.8697 - auc_5: 0.9420
Dataset: 247
125/125 [==============================] - 2s 9ms/step - loss: 0.3087 - binary_accuracy: 0.8667 - auc_6: 0.9434
Dataset: 248
195/195 [==============================] - 3s 7ms/step - loss: 0.4998 - binary_accuracy: 0.7568 - auc_7: 0.8483
Datase

In [ ]:
import pickle
with open('acc.pkl', 'rb') as file:
  avg_acc = pickle.load(file)
for acc in avg_acc:
  print(acc)

0.9698153734207153
0.929977536201477
0.9766781330108643
0.9764435887336731
0.9333206415176392
0.9420354962348938
0.9434300661087036
0.848273515701294
0.9723823070526123
0.9713690280914307
0.9657667279243469
0.9016256332397461
0.9519780874252319
0.9444025754928589
0.912808895111084
0.9212120175361633
0.9556626677513123
0.9057461619377136
0.9325785040855408
0.9636701941490173
0.9369425177574158
0.8146703839302063
0.9284705519676208
0.9394945502281189
0.9058265089988708
0.9603830575942993
0.928296685218811
0.940176248550415
0.9498319029808044
0.9409414529800415
0.9582990407943726
0.9648569226264954
0.9284125566482544
0.954596757888794
0.9770355224609375
0.9454697370529175
0.9491232633590698
0.9296199679374695
0.8906521201133728
0.9601134657859802
0.9738634824752808
0.9127882719039917
0.9622238874435425
0.9423323273658752
0.9422839879989624
0.9509872794151306
0.9292495846748352


In [ ]:
import gc
from keras.models import load_model
i = 0
avg_acc = []
for train_feature,train_label,test_feature,test_label in zip(train_features,train_labels,test_features,test_labels):
  i=i+1
  if i<=287:
    continue
  if i>=290:
    break
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_medium_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.0001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(train_feature,dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=4096,
                              epochs=30,
                              verbose=0,
                              validation_split=0.1,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr,
                                          early_stop]))
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  with open('acc.pkl', 'wb') as file:
    pickle.dump(avg_acc, file)
  gc.collect()

Dataset: 288
140/140 [==============================] - 3s 10ms/step - loss: 0.2734 - binary_accuracy: 0.8886 - auc: 0.9528
Dataset: 289
33/33 [==============================] - 2s 16ms/step - loss: 0.4055 - binary_accuracy: 0.8261 - auc_1: 0.9023


In [ ]:
for acc in avg_acc:
  print(acc)

0.9527779221534729
0.902341365814209


In [ ]:
import gc
from keras.models import load_model
i = 0
avg_acc = []
for train_feature,train_label,test_feature,test_label in zip(train_features,train_labels,test_features,test_labels):
  i=i+1
  if i<=289:
    continue
  if i>340:
    break
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_medium_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.0001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(train_feature,dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=4096,
                              epochs=30,
                              verbose=0,
                              validation_split=0.1,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr,
                                          early_stop]))
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  with open('acc.pkl', 'wb') as file:
    pickle.dump(avg_acc, file)
  gc.collect()

Dataset: 290
72/72 [==============================] - 3s 13ms/step - loss: 0.3021 - binary_accuracy: 0.8603 - auc: 0.9441
Dataset: 291
51/51 [==============================] - 3s 14ms/step - loss: 0.4701 - binary_accuracy: 0.7781 - auc_1: 0.8774
Dataset: 292
41/41 [==============================] - 2s 13ms/step - loss: 0.5359 - binary_accuracy: 0.7543 - auc_2: 0.8526
Dataset: 293
195/195 [==============================] - 3s 8ms/step - loss: 0.2544 - binary_accuracy: 0.8927 - auc_3: 0.9608
Dataset: 294
31/31 [==============================] - 2s 12ms/step - loss: 0.5521 - binary_accuracy: 0.7322 - auc_4: 0.8269
Dataset: 295
66/66 [==============================] - 2s 10ms/step - loss: 0.3135 - binary_accuracy: 0.8567 - auc_5: 0.9401
Dataset: 296
39/39 [==============================] - 2s 11ms/step - loss: 0.4011 - binary_accuracy: 0.8225 - auc_6: 0.9019
Dataset: 297
63/63 [==============================] - 2s 10ms/step - loss: 0.3321 - binary_accuracy: 0.8620 - auc_7: 0.9323
Dataset: 

In [ ]:
import pickle
with open('acc.pkl', 'rb') as file:
  avg_acc = pickle.load(file)
for acc in avg_acc:
  print(acc)

0.9440526962280273
0.8774194717407227
0.8526272773742676
0.9607831835746765
0.8269167542457581
0.9401288032531738
0.9019421339035034
0.9323328733444214
0.9479165077209473
0.9384163022041321
0.843609094619751
0.953217089176178
0.9627081155776978
0.9207045435905457
0.9328941702842712
0.9229215979576111
0.9581671953201294
0.9758070707321167
0.9253115057945251
0.9430683851242065
0.7797423601150513
0.9345324635505676
0.9570850729942322
0.9301390051841736
0.9333649277687073
0.9039618372917175
0.8984732031822205
0.9470553398132324
0.872264564037323
0.941552460193634
0.8714865446090698
0.8403075337409973
0.8741443157196045
0.9501305818557739
0.9396620988845825
0.956638514995575
0.9216238260269165
0.9500912427902222
0.8836885690689087
0.9588618278503418
0.9352270364761353
0.9311678409576416
0.9079235196113586
0.9435986876487732
0.9285023808479309
0.9332229495048523
0.9045760035514832


In [ ]:
import gc
from keras.models import load_model
i = 0
avg_acc = []
for train_feature,train_label,test_feature,test_label in zip(train_features,train_labels,test_features,test_labels):
  i=i+1
  if i<=336:
    continue
  if i>340:
    break
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_medium_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.0001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(train_feature,dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=4096,
                              epochs=30,
                              verbose=0,
                              validation_split=0.1,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr,
                                          early_stop]))
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  with open('acc.pkl', 'wb') as file:
    pickle.dump(avg_acc, file)
  gc.collect()

Dataset: 337
33/33 [==============================] - 2s 17ms/step - loss: 0.2215 - binary_accuracy: 0.9179 - auc: 0.9757
Dataset: 338
29/29 [==============================] - 3s 16ms/step - loss: 0.3018 - binary_accuracy: 0.8830 - auc_1: 0.9445
Dataset: 339
70/70 [==============================] - 2s 10ms/step - loss: 0.2783 - binary_accuracy: 0.8862 - auc_2: 0.9558
Dataset: 340
182/182 [==============================] - 3s 8ms/step - loss: 0.3183 - binary_accuracy: 0.8553 - auc_3: 0.9373


In [ ]:
for acc in avg_acc:
  print(acc)

0.9756680130958557
0.9444981217384338
0.9558305740356445
0.9373381733894348


In [ ]:
import gc
from keras.models import load_model
i = 0
avg_acc = []
for train_feature,train_label,test_feature,test_label in zip(train_features,train_labels,test_features,test_labels):
  i=i+1
  if i<=340:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_medium_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.0001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(train_feature,dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=4096,
                              epochs=30,
                              verbose=0,
                              validation_split=0.1,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr,
                                          early_stop]))
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  with open('acc.pkl', 'wb') as file:
    pickle.dump(avg_acc, file)
  gc.collect()

Dataset: 341
130/130 [==============================] - 3s 9ms/step - loss: 0.2906 - binary_accuracy: 0.8774 - auc: 0.9484
Dataset: 342
74/74 [==============================] - 3s 10ms/step - loss: 0.3678 - binary_accuracy: 0.8364 - auc_1: 0.9194
Dataset: 343
180/180 [==============================] - 3s 8ms/step - loss: 0.3359 - binary_accuracy: 0.8513 - auc_2: 0.9308
Dataset: 344
59/59 [==============================] - 2s 14ms/step - loss: 0.2140 - binary_accuracy: 0.9073 - auc_3: 0.9727
Dataset: 345
144/144 [==============================] - 2s 8ms/step - loss: 0.2703 - binary_accuracy: 0.8834 - auc_4: 0.9556
Dataset: 346
214/214 [==============================] - 3s 8ms/step - loss: 0.3617 - binary_accuracy: 0.8403 - auc_5: 0.9188
Dataset: 347
48/48 [==============================] - 2s 12ms/step - loss: 0.4382 - binary_accuracy: 0.7906 - auc_6: 0.8797
Dataset: 348
149/149 [==============================] - 2s 8ms/step - loss: 0.3666 - binary_accuracy: 0.8351 - auc_7: 0.9159
Datas

In [ ]:
import pickle
with open('acc.pkl', 'rb') as file:
  avg_acc = pickle.load(file)
for acc in avg_acc:
  print(acc)

0.9483782052993774
0.9194276928901672
0.9308078289031982
0.9726541042327881
0.9556204080581665
0.9187718033790588
0.8797110319137573
0.9158514142036438
0.8764652013778687
0.9462387561798096
0.9437766075134277
0.9319170117378235
0.9581570029258728
0.9761252403259277
0.8497235178947449
0.8750702142715454
0.9396313428878784
0.9459580779075623
0.9620512127876282
0.9800726771354675
0.9765676259994507
0.9628422856330872
0.8818608522415161
0.9459258913993835
0.9234911799430847
0.9504748582839966
0.9395356774330139
0.9301621913909912
0.899476945400238
0.9699118137359619
0.971352219581604
0.9116626381874084
0.9400287866592407
0.9124937653541565
0.9440792798995972
0.9342581033706665
0.9707627892494202
0.980764627456665
0.9600120186805725
0.9560553431510925
0.948648989200592
0.9432477951049805
0.9553588628768921
0.9253710508346558
0.9336678385734558
0.973364531993866
0.9620975852012634


In [ ]:
import gc
from keras.models import load_model
i = 0
avg_acc = []
for train_feature,train_label,test_feature,test_label in zip(train_features,train_labels,test_features,test_labels):
  i=i+1
  if i<388:
    continue
  print("Dataset:",i)
  CBLANE = load_model("CBLANE_medium_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.0001),
                              metrics=[BinaryAccuracy(),
                                        AUC(),
                                        ]
                              )
  history = CBLANE.fit(tf.constant(train_feature,dtype=tf.bool),
                              tf.constant(train_label,dtype=tf.bool),
                              batch_size=4096,
                              epochs=30,
                              verbose=0,
                              validation_split=0.1,
                              validation_batch_size=4096,
                              callbacks=([reduce_lr,
                                          early_stop]))
  data = CBLANE.evaluate(test_feature,test_label,verbose=1)
  avg_acc.append(data[2])
  with open('acc.pkl', 'wb') as file:
    pickle.dump(avg_acc, file)
  gc.collect()

Dataset: 388
198/198 [==============================] - 4s 10ms/step - loss: 0.4095 - binary_accuracy: 0.8118 - auc: 0.8984
Dataset: 389
60/60 [==============================] - 2s 10ms/step - loss: 0.1889 - binary_accuracy: 0.9212 - auc_1: 0.9799


In [ ]:
import pickle
with open('acc.pkl', 'rb') as file:
  avg_acc = pickle.load(file)
for acc in avg_acc:
  print(acc)

0.8984125852584839
0.9798734784126282


# Small Dataset

## Pre Fine-Tuning

In [ ]:
avg_acc = []
i=0
for trainset,testset in train_test:
  i+=1
  print("iteration",i)
  train_features = np.unpackbits(trainset[0],axis=2,count=4)
  train_labels = np.unpackbits(trainset[1],axis=-1,count=train_features.shape[0])
  test_features =  np.unpackbits(testset[0],axis=2,count=4)
  test_labels = np.unpackbits(testset[1],axis=-1,count=test_features.shape[0])
  CBLANE = load_model("CBLANE_small_dataset.keras")
  data = CBLANE.evaluate(test_features,test_labels,verbose=1,batch_size=1024)
  avg_acc.append(data[4])
  print(np.mean(avg_acc))

iteration 1
1/1 [==============================] - 2s 2s/step - loss: 0.3396 - binary_accuracy: 0.8526 - precision: 0.8269 - recall: 0.8699 - auc: 0.9334 - sensitivity_at_specificity: 0.9777 - specificity_at_sensitivity: 0.9801
0.9333571791648865
iteration 2
1/1 [==============================] - 2s 2s/step - loss: 0.3898 - binary_accuracy: 0.8147 - precision: 0.8080 - recall: 0.8080 - auc: 0.9038 - sensitivity_at_specificity: 0.9440 - specificity_at_sensitivity: 0.9851
0.9185741245746613
iteration 3
1/1 [==============================] - 2s 2s/step - loss: 0.3760 - binary_accuracy: 0.8100 - precision: 0.8361 - recall: 0.7669 - auc: 0.9106 - sensitivity_at_specificity: 0.9693 - specificity_at_sensitivity: 0.9849
0.9159220655759176
iteration 4


1/1 [==============================] - 2s 2s/step - loss: 0.4942 - binary_accuracy: 0.7646 - precision: 0.8308 - recall: 0.6486 - auc: 0.8575 - sensitivity_at_specificity: 0.9129 - specificity_at_sensitivity: 0.9402
0.9013119041919708
iteration 5


1/1 [==============================] - 2s 2s/step - loss: 0.3360 - binary_accuracy: 0.8732 - precision: 0.8727 - recall: 0.8807 - auc: 0.9329 - sensitivity_at_specificity: 0.9358 - specificity_at_sensitivity: 0.9712
0.9076321005821228
iteration 6
1/1 [==============================] - 2s 2s/step - loss: 0.3472 - binary_accuracy: 0.8463 - precision: 0.8680 - recall: 0.8251 - auc: 0.9233 - sensitivity_at_specificity: 0.9696 - specificity_at_sensitivity: 0.9960
0.9102464516957601
iteration 7
1/1 [==============================] - 3s 3s/step - loss: 0.4259 - binary_accuracy: 0.8068 - precision: 0.8857 - recall: 0.7045 - auc: 0.8967 - sensitivity_at_specificity: 0.9545 - specificity_at_sensitivity: 0.9773
0.908310421875545
iteration 8
1/1 [==============================] - 2s 2s/step - loss: 0.5617 - binary_accuracy: 0.7419 - precision: 0.8227 - recall: 0.6013 - auc: 0.8335 - sensitivity_at_specificity: 0.9037 - specificity_at_sensitivity: 0.9333
0.8989574015140533
iteration 9
1/1 [========

## Fine-Tuning

In [ ]:
from keras.models import load_model
CBLANE = load_model("CBLANE.keras")
CBLANE.compile(loss='binary_crossentropy',
                             optimizer=Adam(learning_rate=0.0001),
                             metrics=[BinaryAccuracy(),
                                      Precision(),
                                      Recall(),
                                      AUC(),
                                      SensitivityAtSpecificity(0.5),
                                      SpecificityAtSensitivity(0.5),
                                      ]
                             )
history = CBLANE.fit(tf.constant(small_train_features,dtype=tf.bool),
                             tf.constant(small_train_labels,dtype=tf.bool),
                             batch_size=128,
                             epochs=10,
                             verbose=1,
                            #  validation_split=0.1,
                            #  validation_batch_size=4096,
                             callbacks=([reduce_lr]))

Epoch 1/10
741/741 [==============================] - 43s 32ms/step - loss: 0.4211 - binary_accuracy: 0.8013 - precision: 0.8181 - recall: 0.7728 - auc: 0.8854 - sensitivity_at_specificity: 0.9429 - specificity_at_sensitivity: 0.9695 - lr: 1.0000e-04
Epoch 2/10
741/741 [==============================] - 23s 31ms/step - loss: 0.3845 - binary_accuracy: 0.8221 - precision: 0.8404 - recall: 0.7934 - auc: 0.9052 - sensitivity_at_specificity: 0.9568 - specificity_at_sensitivity: 0.9820 - lr: 1.0000e-04
Epoch 3/10
741/741 [==============================] - 23s 31ms/step - loss: 0.3597 - binary_accuracy: 0.8353 - precision: 0.8533 - recall: 0.8083 - auc: 0.9175 - sensitivity_at_specificity: 0.9656 - specificity_at_sensitivity: 0.9874 - lr: 1.0000e-04
Epoch 4/10
741/741 [==============================] - 23s 31ms/step - loss: 0.3370 - binary_accuracy: 0.8479 - precision: 0.8643 - recall: 0.8237 - auc: 0.9280 - sensitivity_at_specificity: 0.9714 - specificity_at_sensitivity: 0.9912 - lr: 1.0000e

## Post Fine-Tuning

In [ ]:
avg_acc = []
i=0

for trainset,testset in train_test:
  i+=1
  print("Dataset:",i)
  train_features = np.unpackbits(trainset[0],axis=2,count=4)
  train_labels = np.unpackbits(trainset[1],axis=-1,count=train_features.shape[0])
  test_features =  np.unpackbits(testset[0],axis=2,count=4)
  test_labels = np.unpackbits(testset[1],axis=-1,count=test_features.shape[0])
  CBLANE=load_model("CBLANE_small_dataset.keras")
  CBLANE.compile(loss='binary_crossentropy',
                              optimizer=Adam(learning_rate=0.001),
                              metrics=[BinaryAccuracy(),
                                        AUC()])
  history = CBLANE.fit(tf.constant(train_features,dtype=tf.bool),
                   tf.constant(train_labels,dtype=tf.bool),
                   batch_size=128,
                   epochs=20,
                   verbose=0,
                   validation_split=0.1,
                   callbacks=[reduce_lr,
                              early_stop]
                         )
  data = CBLANE.evaluate(test_features,test_labels,verbose=1)
  avg_acc.append(data[2])
print(np.mean(avg_acc))

Dataset: 1
18/18 [==============================] - 2s 7ms/step - loss: 0.3328 - binary_accuracy: 0.8649 - auc_4: 0.9456
Dataset: 2
9/9 [==============================] - 2s 11ms/step - loss: 0.3224 - binary_accuracy: 0.8456 - auc_5: 0.9328
Dataset: 3
21/21 [==============================] - 1s 8ms/step - loss: 0.3291 - binary_accuracy: 0.8511 - auc_6: 0.9328
Dataset: 4
22/22 [==============================] - 1s 7ms/step - loss: 0.5061 - binary_accuracy: 0.7558 - auc_7: 0.8394
Dataset: 5
7/7 [==============================] - 2s 49ms/step - loss: 0.3345 - binary_accuracy: 0.9108 - auc_8: 0.9586
Dataset: 6
17/17 [==============================] - 2s 17ms/step - loss: 0.3945 - binary_accuracy: 0.8502 - auc_9: 0.9255
Dataset: 7
3/3 [==============================] - 1s 127ms/step - loss: 0.4743 - binary_accuracy: 0.7614 - auc_10: 0.8920
Dataset: 8
20/20 [==============================] - 2s 23ms/step - loss: 0.5171 - binary_accuracy: 0.7500 - auc_11: 0.8337
Dataset: 9
17/17 [============

0.9455779790878296
0.9328060150146484
0.9327787160873413
0.839433491230011
0.9586274027824402
0.9255146980285645
0.8920454978942871
0.8337130546569824
0.905036211013794
0.965635359287262
0.9437276124954224
0.9720394611358643
0.8044293522834778
0.8463839888572693
0.9259503483772278
0.7305905222892761
0.755374014377594
0.8684654831886292
0.9248650074005127
0.8942936062812805
0.9236869812011719
0.9544438719749451
0.9606000185012817
0.9571592807769775
0.9089686870574951
0.8868129849433899
0.9710796475410461
0.9432327747344971
0.9750210046768188
0.964488685131073
0.9016019105911255
0.958983838558197
0.8313953876495361
0.8606799840927124
0.7945946455001831
0.8654711246490479
0.9581191539764404
0.9466153979301453
0.8712252974510193
0.8930299878120422
0.8776687979698181
0.968316376209259
0.8935844302177429
0.9268602728843689
0.879302442073822
0.8703249096870422
0.8615824580192566
0.8509351015090942
0.8925833702087402
0.8328174948692322
0.9518271088600159
0.9602565765380859
0.8997674584388733
0